In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.auto import tqdm

from scipy.stats import wilcoxon
import warnings


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
exchange = "cboe"
models = ["gbm", "classical"] # ["gbm", "classical"]
subset = "test" # "all"
strategy = "supervised"  # "transfer"


In [3]:
key = f"{exchange}_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact cboe_supervised_raw:latest, 1937.19MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_gbm_supervised_test:latest, 56.26MB. 1 files... 
/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.5
wandb: Downloading large artifact cboe_classical_supervised_test:latest, 212.93MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )


results = []
for i, model in tqdm(enumerate(models)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=models)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:01,  1.11it/s]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        ("gbm", "gbm(classical-size)"),
        ("gbm", "gbm(ml)"),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [6]:
results_data


gbm                             classical            \
         gbm(classical) gbm(classical-size) gbm(ml)  tick(ex) quote(ex)   
index                                                                     
29930521              1                   1       1       1.0      -1.0   
29894717              1                   1       1      -1.0       1.0   
29909660              1                   1       1      -1.0      -1.0   
29939322             -1                  -1      -1      -1.0      -1.0   
29883818             -1                   1       1      -1.0       1.0   
...                 ...                 ...     ...       ...       ...   
37155409              1                  -1       1       1.0      -1.0   
37155410              1                   1       1       1.0       1.0   
37104659             -1                  -1      -1      -1.0       1.0   
37108365              1                   1       1      -1.0      -1.0   
37155411              1                   1       1      -1.0      -1.0   

                                  \
         lr(ex) emo(ex) clnv(ex)   
index                              
29930521   -1.0    -1.0     -1.0   
29894717    1.0     1.0      1.0   
29909660   -1.0    -1.0     -1.0   
29939322   -1.0     1.0     -1.0   
29883818    1.0     1.0      1.0   
...         ...     ...      ...   
37155409   -1.0    -1.0     -1.0   
37155410    1.0     1.0      1.0   
37104659    1.0     1.0      1.0   
37108365   -1.0    -1.0     -1.0   
37155411   -1.0    -1.0     -1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
index                                                                                   
29930521                                                1.0                             
29894717                                                1.0                             
29909660                                               -1.0                             
29939322                                               -1.0                             
29883818                                                1.0                             
...                                                     ...                             
37155409                                               -1.0                             
37155410                                                1.0                             
37104659                                               -1.0                             
37108365                                                1.0                             
37155411                                                1.0                             

[7431766 rows x 9 columns]

### Robustness Checks

In [7]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock options",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7 c",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        ((mid - half_spread) < trade_price) & (trade_price < (mid + half_spread)),
        1,  # inside
        np.where(
            (trade_price == (mid - half_spread)) | ((mid + half_spread) == trade_price),
            2,  # at quotes
            np.where(
                (trade_price < (mid - half_spread))
                | ((mid + half_spread) > trade_price),
                3,
                4,
            ),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [8]:
X_print.head(20)


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid
index,,,,,,,,,,
29930521,1,0.02,C,Others,"(0,1]",2016,<= 1,(0.7-0.9],at quotes,0.025
29894717,1,0.27,P,Stock options,"(5,11]",2016,<= 1,(0.9-1.1],inside,0.225
29909660,1,0.20,C,Stock options,"(0,1]",2016,(1-2],(0.7-0.9],inside,0.350
29939322,-1,0.12,C,Others,"(0,1]",2016,<= 1,(0.9-1.1],inside,0.135
29883818,1,1.85,C,Stock options,"(3,5]",2016,(1-2],(0.9-1.1],at quotes,1.725
29934231,1,4.50,C,Stock options,>11,2016,> 12,> 1.3,at quotes,4.700
29938519,1,1.20,P,Stock options,"(3,5]",2016,(2-3],(1.1-1.3],at quotes,1.100
29939396,-1,3.55,P,Stock options,>11,2016,(3-6],(0.7-0.9],unknown,NaN
29936750,1,0.55,P,Others,"(1,3]",2016,<= 1,(0.7-0.9],inside,0.565


In [9]:
X_print = pd.concat([X_print, results_data], axis=1)


In [10]:
X_print.head()


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,
29930521,1,0.02,C,Others,"(0,1]",2016,<= 1,(0.7-0.9],at quotes,0.025,1,1,1,1.0,-1.0,-1.0,-1.0,-1.0,1.0
29894717,1,0.27,P,Stock options,"(5,11]",2016,<= 1,(0.9-1.1],inside,0.225,1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0
29909660,1,0.20,C,Stock options,"(0,1]",2016,(1-2],(0.7-0.9],inside,0.350,1,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
29939322,-1,0.12,C,Others,"(0,1]",2016,<= 1,(0.9-1.1],inside,0.135,-1,-1,-1,-1.0,-1.0,-1.0,1.0,-1.0,-1.0
29883818,1,1.85,C,Stock options,"(3,5]",2016,(1-2],(0.9-1.1],at quotes,1.725,-1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0


## Results Set Generation

In [11]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    # "(": "(",  # put interval start in math env
    # "]": "]",  # put interval end in math env
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [137]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)
        
    if bold:
        res = (res.apply(
            highlight_max, props="font-weight:bold;", axis=bold_axis
        )
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        # .hide(axis="index", level=0)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [13]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


In [14]:
results_data.columns.tolist()


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [15]:
X_print.head()

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,
29930521,1,0.02,C,Others,"(0,1]",2016,<= 1,(0.7-0.9],at quotes,0.025,1,1,1,1.0,-1.0,-1.0,-1.0,-1.0,1.0
29894717,1,0.27,P,Stock options,"(5,11]",2016,<= 1,(0.9-1.1],inside,0.225,1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0
29909660,1,0.20,C,Stock options,"(0,1]",2016,(1-2],(0.7-0.9],inside,0.350,1,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
29939322,-1,0.12,C,Others,"(0,1]",2016,<= 1,(0.9-1.1],inside,0.135,-1,-1,-1,-1.0,-1.0,-1.0,1.0,-1.0,-1.0
29883818,1,1.85,C,Stock options,"(3,5]",2016,(1-2],(0.9-1.1],at quotes,1.725,-1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
X_print[X_print["prox_q_binned"]=="unknown"]

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,
29939396,-1,3.550000,P,Stock options,>11,2016,(3-6],(0.7-0.9],unknown,NaN,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
29935218,1,2.570000,C,Stock options,"(0,1]",2016,<= 1,(0.9-1.1],unknown,NaN,-1,1,1,1.0,-1.0,-1.0,-1.0,-1.0,1.0
29783822,-1,0.670000,C,Others,"(0,1]",2016,(3-6],(0.9-1.1],unknown,0.640000,1,-1,-1,-1.0,1.0,1.0,1.0,1.0,-1.0
29655576,-1,291.799988,P,Index option,"(0,1]",2016,(3-6],(0.9-1.1],unknown,NaN,1,1,-1,1.0,1.0,1.0,1.0,1.0,1.0
29934780,-1,3.650000,C,Others,"(0,1]",2016,<= 1,(0.9-1.1],unknown,NaN,-1,-1,-1,1.0,1.0,1.0,1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37151178,-1,0.400000,C,Stock options,"(0,1]",2017,(3-6],(0.9-1.1],unknown,0.380000,-1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0
37152504,1,24.420000,P,Others,"(0,1]",2017,(6-12],(0.9-1.1],unknown,24.264999,-1,1,1,-1.0,1.0,1.0,1.0,1.0,1.0
37155404,-1,0.800000,C,Index option,>11,2017,(3-6],<= 0.7 c,unknown,0.775000,-1,-1,-1,-1.0,1.0,1.0,1.0,1.0,1.0


## Accuracy Calculation

In [17]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        acc_tot = accuracy_score(
            X_print["buy_sell"].astype("int8"), X_print[classifier]
        )

        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 7/7 [02:17<00:00, 19.65s/it]


In [18]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [19]:
master


gbm                                 \
                                gbm(classical) gbm(classical-size)    gbm(ml)   
Option Type       C                  69.929332           74.756527  77.061559   
                  P                  70.760558           74.645476  76.841392   
                  all                70.308753           74.705837  76.961061   
Security Type     Index option       68.338618           72.520754  76.527357   
                  Others             71.493258           75.251770  77.334693   
                  Stock options      69.979251           74.662893  76.841073   
                  all                70.308753           74.705837  76.961061   
Trade Size        (0,1]              68.092407           72.620323  75.319963   
                  (1,3]              69.205331           73.683006  76.151300   
                  (3,5]              69.687525           74.403358  76.566618   
                  (5,11]             70.951622           75.010330  77.050201   
                  >11                74.590981           78.800083  80.441287   
                  all                70.308753           74.705837  76.961061   
Year              2016               70.843835           75.275833  77.800924   
                  2017               70.162344           74.549874  76.731258   
                  all                70.308753           74.705837  76.961061   
Time to Maturity  <= 1               70.555488           74.287969  76.463490   
                  (1-2]              71.600389           75.493369  77.950948   
                  (2-3]              71.099750           75.899742  78.066591   
                  (3-6]              70.271222           76.207334  78.259998   
                  (6-12]             69.303696           74.826243  77.466154   
                  > 12               64.125740           72.253504  75.039542   
                  all                70.308753           74.705837  76.961061   
Moneyness         <= 0.7 c           71.619038           77.268103  79.754269   
                  (0.7-0.9]          71.104891           75.173820  77.723148   
                  (0.9-1.1]          70.844434           74.973394  77.042594   
                  (1.1-1.3]          63.309037           70.194063  72.921703   
                  > 1.3              63.025877           69.954130  74.179209   
                  all                70.308753           74.705837  76.961061   
Location to Quote at mid             62.321390           67.947788  70.443174   
                  inside             70.691264           73.931190  75.856277   
                  at quotes          74.033610           83.833855  87.630890   
                  outside            70.840165           82.245906  87.144984   
                  unknown            76.922608           84.666707  88.041715   
                  all                70.308753           74.705837  76.961061   

                                 classical                                   \
                                  tick(ex)  quote(ex)     lr(ex)    emo(ex)   
Option Type       C              48.955333  60.850513  60.553543  48.243383   
                  P              49.490788  60.934859  60.747611  48.077527   
                  all            49.199746  60.889013  60.642127  48.167676   
Security Type     Index option   48.487310  50.334319  50.260448  39.699766   
                  Others         49.153068  65.261752  64.922796  50.091092   
                  Stock options  49.280091  59.952437  59.729468  48.081917   
                  all            49.199746  60.889013  60.642127  48.167676   
Trade Size        (0,1]          49.041394  58.033645  57.900579  46.664499   
                  (1,3]          49.214207  61.326282  61.092161  48.401654   
                  (3,5]          48.820721  61.082123  60.806340  48.227728   
                  (5,11]         49.106302  61.512827  61.232605  48.207533   
                  

In [20]:
master.style.pipe(
    set_tex_style, caption=("master-long", "master-short"), label=f"{key}-master", bold_axis=0
)


## Effective Spread 🚧

In [21]:
classifiers


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [22]:
eff_dfs = []


def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = (
        effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")
        * 100
    )

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")

    return pd.Series(
        {
            "nominal": nom,
            "rel": rel,
            # 'statistic':wilcoxon_res.statistic,
            # 'pvalue':wilcoxon_res.pvalue
        }
    )


for criterion in tqdm(criterions):
    results = []

    for classifier in tqdm(classifiers):
        res = X_print.groupby([criterion])[
            ["TRADE_PRICE", "mid", classifier, "buy_sell"]
        ].apply(stats, classifier)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}-eff-spread",
    )

    # store all result sets for later use
    eff_dfs.append(result_df)


100%|██████████| 7/7 [00:45<00:00,  6.47s/it]


In [23]:
master_eff_spread = pd.concat(eff_dfs, axis=1, keys=LUT_INDEX.values()).T


In [24]:
master_eff_spread


gbm                 \
                                gbm(classical)                  
                                       nominal            rel   
Option Type       C                   0.037183  5.568612e+302   
                  P                   0.023943  -1.961121e+01   
Security Type     Index option       -0.008027   1.692972e+00   
                  Others              0.024162   6.773617e+00   
                  Stock options       0.037412  9.112811e+302   
Trade Size        (0,1]               0.027974  2.176855e+303   
                  (1,3]               0.032963   1.692723e+00   
                  (3,5]               0.032180   5.762984e+00   
                  (5,11]              0.030742   6.143302e+00   
                  >11                 0.033330   7.227341e+00   
Year              2016                0.028062   4.682052e+00   
                  2017                0.031980   6.174744e+00   
Time to Maturity  <= 1                0.026884   7.550903e+00   
                  (1-2]               0.032591  7.268720e+303   
                  (2-3]               0.025268  4.398703e+303   
                  (3-6]               0.036415   2.789134e+00   
                  (6-12]              0.036176   2.444805e+00   
                  > 12                0.069044  6.471238e+303   
Moneyness         <= 0.7 c            0.006647   9.450880e+00   
                  (0.7-0.9]           0.020980   9.551971e+00   
                  (0.9-1.1]           0.032613  4.306566e+302   
                  (1.1-1.3]           0.047263   8.209339e-01   
                  > 1.3               0.066832   3.862266e-01   
Location to Quote at mid              0.000000   0.000000e+00   
                  inside              0.059250   6.556484e+00   
                  at quotes          -0.107027   8.114832e+00   
                  outside            -0.089660  -4.672395e+00   
                  unknown            -0.036567            inf   

                                                                              \
                                gbm(classical-size)                  gbm(ml)   
                                            nominal            rel   nominal   
Option Type       C                        0.021744  5.568612e+302  0.017214   
                  P                        0.013383  -2.088343e+01  0.011878   
Security Type     Index option             0.001682   2.497100e+00 -0.013722   
                  Others                   0.011002   5.665873e+00  0.013282   
                  Stock options            0.022223  9.112811e+302  0.017838   
Trade Size        (0,1]                    0.005834  2.176855e+303  0.005084   
                  (1,3]                    0.022711   3.988364e-01  0.023089   
                  (3,5]                    0.025546   4.717831e+00  0.015254   
                  (5,11]                   0.020257   5.290992e+00  0.015428   
                  >11                      0.022527   6.695074e+00  0.018634   
Year              2016                     0.009825   2.982258e+00  0.004517   
                  2017                     0.020144   4.983556e+00  0.017585   
Time to Maturity  <= 1                     0.017995   6.051896e+00  0.013380   
                  (1-2]                    0.018591  7.268720e+303  0.017976   
                  (2-3]                    0.006083  4.398703e+303  0.012748   
                  (3-6]                    0.019798   1.832172e+00  0.022349   
                  (6-12]                   0.022991   1.339052e+00  0.028555   
                  > 12                     0.021152  6.471238e+303 -0.010888   
Moneyness         <= 0.7 c                -0.006295   6.665789e+00 -0.022315   
                  (0.7-0.9]                0.008382   7.357694e+00  0.000125   
                  (0.9-1.1]                0.021276  4.306566e+302  0.023823   
                  (1.1-1.3]                0.026950   4.394947e-01 -0.008560   
                  > 1.3    

In [25]:
master_eff_spread.style.pipe(
    set_tex_style,
    caption=("master-short", "master-long"),
    label=f"{key}-master-eff-spread",
    bold_axis=0,
)


## Change in Parenthesis 🅾️

```latex
# https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
\begin{table}
    \centering
    \caption{test of combination with change}
    \label{tab:combo}
    \begin{tabular}{lSSSSSSSS}
        \toprule
        {} & \multicolumn{2}{l}{Index option} & \multicolumn{2}{l}{Others} & \multicolumn{2}{l}{Stock options} & \multicolumn{2}{l}{all} \\
        \midrule
        classical-size & 1.0 & \parl-56.42\parr & 2.0 & \parl-74.35\parr & -73.5 & \parl-143.93\parr & 5.0 & \parl-67.33\parr \\
        \bottomrule
        \end{tabular}
\end{table}
```

In [127]:
master

gbm                                 \
                                gbm(classical) gbm(classical-size)    gbm(ml)   
Option Type       C                  69.929332           74.756527  77.061559   
                  P                  70.760558           74.645476  76.841392   
                  all                70.308753           74.705837  76.961061   
Security Type     Index option       68.338618           72.520754  76.527357   
                  Others             71.493258           75.251770  77.334693   
                  Stock options      69.979251           74.662893  76.841073   
                  all                70.308753           74.705837  76.961061   
Trade Size        (0,1]              68.092407           72.620323  75.319963   
                  (1,3]              69.205331           73.683006  76.151300   
                  (3,5]              69.687525           74.403358  76.566618   
                  (5,11]             70.951622           75.010330  77.050201   
                  >11                74.590981           78.800083  80.441287   
                  all                70.308753           74.705837  76.961061   
Year              2016               70.843835           75.275833  77.800924   
                  2017               70.162344           74.549874  76.731258   
                  all                70.308753           74.705837  76.961061   
Time to Maturity  <= 1               70.555488           74.287969  76.463490   
                  (1-2]              71.600389           75.493369  77.950948   
                  (2-3]              71.099750           75.899742  78.066591   
                  (3-6]              70.271222           76.207334  78.259998   
                  (6-12]             69.303696           74.826243  77.466154   
                  > 12               64.125740           72.253504  75.039542   
                  all                70.308753           74.705837  76.961061   
Moneyness         <= 0.7 c           71.619038           77.268103  79.754269   
                  (0.7-0.9]          71.104891           75.173820  77.723148   
                  (0.9-1.1]          70.844434           74.973394  77.042594   
                  (1.1-1.3]          63.309037           70.194063  72.921703   
                  > 1.3              63.025877           69.954130  74.179209   
                  all                70.308753           74.705837  76.961061   
Location to Quote at mid             62.321390           67.947788  70.443174   
                  inside             70.691264           73.931190  75.856277   
                  at quotes          74.033610           83.833855  87.630890   
                  outside            70.840165           82.245906  87.144984   
                  unknown            76.922608           84.666707  88.041715   
                  all                70.308753           74.705837  76.961061   

                                 classical                                   \
                                  tick(ex)  quote(ex)     lr(ex)    emo(ex)   
Option Type       C              48.955333  60.850513  60.553543  48.243383   
                  P              49.490788  60.934859  60.747611  48.077527   
                  all            49.199746  60.889013  60.642127  48.167676   
Security Type     Index option   48.487310  50.334319  50.260448  39.699766   
                  Others         49.153068  65.261752  64.922796  50.091092   
                  Stock options  49.280091  59.952437  59.729468  48.081917   
                  all            49.199746  60.889013  60.642127  48.167676   
Trade Size        (0,1]          49.041394  58.033645  57.900579  46.664499   
                  (1,3]          49.214207  61.326282  61.092161  48.401654   
                  (3,5]          48.820721  61.082123  60.806340  48.227728   
                  (5,11]         49.106302  61.512827  61.232605  48.207533   
                  

In [133]:
base = master[[("classical","quote(ex)"),("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)"), ("classical", "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)")]]
revised = master[[("gbm","gbm(classical)"),("gbm","gbm(classical-size)"), ("gbm", "gbm(ml)")]]

In [134]:

def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    
    
    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (
            "\parl" + (combo[mul_col] - base.iloc[:,i]).round(2).astype(str) + "\parr"
        )
        # print(combo.shape)
        # # define custom brackets that are not parsed by sunitx
        # combo.iloc[:,[(mul_col[0], "pm")]] = 
        # # sort to group together columns
        # combo.sort_index(axis=1, inplace=True)
        # cols.append(diff)
        combo.sort_index(axis=1, inplace=True)
        
    return combo


In [135]:
diff = combine_results(revised, base)

diff.style.to_latex(
            f"diff-{key}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label=f"tab:diff-{key}",
            caption=(f"long-diff-{key}", f"short-diff-{key}"),
            convert_css=True,
        )

In [136]:
diff


gbm(classical)                   \
                                           nom               pm   
Option Type       C                  69.929332   \parl9.08\parr   
                  P                  70.760558   \parl9.83\parr   
                  all                70.308753   \parl9.42\parr   
Security Type     Index option       68.338618   \parl18.0\parr   
                  Others             71.493258   \parl6.23\parr   
                  Stock options      69.979251  \parl10.03\parr   
                  all                70.308753   \parl9.42\parr   
Trade Size        (0,1]              68.092407  \parl10.06\parr   
                  (1,3]              69.205331   \parl7.88\parr   
                  (3,5]              69.687525   \parl8.61\parr   
                  (5,11]             70.951622   \parl9.44\parr   
                  >11                74.590981  \parl10.77\parr   
                  all                70.308753   \parl9.42\parr   
Year              2016               70.843835  \parl10.22\parr   
                  2017               70.162344    \parl9.2\parr   
                  all                70.308753   \parl9.42\parr   
Time to Maturity  <= 1               70.555488   \parl8.78\parr   
                  (1-2]              71.600389  \parl10.18\parr   
                  (2-3]              71.099750  \parl10.03\parr   
                  (3-6]              70.271222  \parl10.17\parr   
                  (6-12]             69.303696  \parl10.18\parr   
                  > 12               64.125740  \parl11.62\parr   
                  all                70.308753   \parl9.42\parr   
Moneyness         <= 0.7 c           71.619038  \parl16.74\parr   
                  (0.7-0.9]          71.104891  \parl12.06\parr   
                  (0.9-1.1]          70.844434   \parl8.46\parr   
                  (1.1-1.3]          63.309037   \parl7.79\parr   
                  > 1.3              63.025877  \parl10.63\parr   
                  all                70.308753   \parl9.42\parr   
Location to Quote at mid             62.321390  \parl12.49\parr   
                  inside             70.691264   \parl3.75\parr   
                  at quotes          74.033610  \parl38.41\parr   
                  outside            70.840165  \parl31.95\parr   
                  unknown            76.922608  \parl41.76\parr   
                  all                70.308753   \parl9.42\parr   

                                gbm(classical-size)                   \
                                                nom               pm   
Option Type       C                       74.756527   \parl8.64\parr   
                  P                       74.645476   \parl7.63\parr   
                  all                     74.705837   \parl8.18\parr   
Security Type     Index option            72.520754   \parl4.84\parr   
                  Others                  75.251770   \parl6.69\parr   
                  Stock options           74.662893   \parl9.09\parr   
                  all                     74.705837   \parl8.18\parr   
Trade Size        (0,1]                   72.620323   \parl8.55\parr   
                  (1,3]                   73.683006   \parl8.32\parr   
                  (3,5]                   74.403358   \parl8.09\parr   
                  (5,11]                  75.010330   \parl8.65\parr   
                  >11                     78.800083   \parl7.12\parr   
                  all                     74.705837   \parl8.18\parr   
Year              2016                    75.275833  \parl10.81\parr   
                  2017                    74.549874   \parl7.46\parr   
                  all                     74.705837   \parl8.18\parr   
Time to Maturity  <= 1                    74.287969   \parl7.67\parr   
                  (1-2]                   75.493369   \parl8.49\parr   
                  (2-3]                   75.899742   \parl8.31\parr   
                  (3-6]                 